# ALERT SYSTEM

ADX: https://www.investopedia.com/articles/trading/07/adx-trend-indicator.asp

Cron Job: https://crontab.guru/
*/2 * * * *

In [2]:
import pandas as pd
import pandas_ta as ta
import ccxt, yfinance
import requests

In [3]:
exchange = ccxt.binance()

In [4]:
symbols = pd.read_csv("symbols.csv")
symbols = pd.DataFrame(symbols, columns = ["symbol"])
symbols = symbols["symbol"].values.tolist()

In [19]:
for symbol in symbols:
    bars = exchange.fetch_ohlcv(symbol, timeframe = "5m", limit = 500)
    df = pd.DataFrame(bars, columns = ["time", "open", "high", "low", "close", "volume"])
    cap_time = pd.to_datetime(df["time"],  unit='ms')[1]
    
    # technical indicators
    adx = df.ta.adx()
    macd = df.ta.macd(fast = 14, slow = 28)
    rsi = df.ta.rsi()

    # create composite df
    df = pd.concat([df, adx, macd, rsi], axis = 1)

    # grab the last value
    last_row = df.iloc[-1]

    # hook to discord channel
    WEBHOOK_URL = "https://discord.com/api/webhooks/982327105041875004/w_ume6BIxlHe3C9jgRUrUXWSFn3oiLcvpb2118vsNh8iZdRGHxD5LqYDer2cGwsjd_sL"

    # logic
    if last_row["ADX_14"] > 25:
        if last_row["DMP_14"] > last_row["DMN_14"]:
            message = f"{cap_time} STRONG UP TREND: {symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"
        if last_row["DMN_14"] > last_row["DMP_14"]:
            message = f"{cap_time} STRONG DOWN TREND:{symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"

        payload = {
            "username" : "alertbot",
            "content" : message
        }

        print(message)
        #requests.post(WEBHOOK_URL, json = payload)

    if last_row["ADX_14"] < 25:
        message = f"{cap_time} no trend:{symbol}: The ADX is {last_row['ADX_14']: .2f} +DI {last_row['DMP_14']: .2f} -DI {last_row['DMN_14']: .2f}"

        payload = {
            "username" : "alertbot",
            "content" : message
        }
        
        print(message)
        #requests.post(WEBHOOK_URL, json = payload)

2022-06-02 00:05:00 no trend:ETH/USDT: The ADX is  11.41 +DI  16.23 -DI  21.53
2022-06-02 00:05:00 no trend:BTC/USDT: The ADX is  14.90 +DI  23.94 -DI  15.56
2022-06-02 00:05:00 no trend:SOL/USDT: The ADX is  10.32 +DI  18.33 -DI  16.10
2022-06-02 00:05:00 no trend:AVAX/USDT: The ADX is  24.82 +DI  22.58 -DI  14.63
